<a href="https://colab.research.google.com/github/monoramasn/EDACC_WHISPER_MMS_SEAMLESS/blob/main/edacc_whisper_erm_sd_dro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install -U accelerate
!pip install evaluate
!pip install librosa
!pip install jiwer
!pip install --upgrade transformers bitsandbytes datasets torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succes

In [ ]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import argparse
import evaluate
from scipy import signal
import numpy as np
from dataclasses import dataclass
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from typing import Any, Dict, List, Union
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict, Audio, load_from_disk, concatenate_datasets, load_metric
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, TrainingArguments

In [ ]:
import os
import torchaudio
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import WhisperFeatureExtractor, WhisperProcessor, WhisperTokenizer, WhisperForConditionalGeneration, TrainerCallback, TrainingArguments, Trainer
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import logging

In [ ]:
import os
import torchaudio
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load datasets
edacc_dev = load_dataset("edinburghcstr/edacc", split="validation")
edacc_test = load_dataset("edinburghcstr/edacc", split="test")

# Create directories for each language in the test, train, and validation sets
base_dir = "organized_data"
os.makedirs(base_dir, exist_ok=True)

languages = edacc_test.unique("l1")
for split in ['test', 'train', 'validation']:
    for lang in languages:
        os.makedirs(os.path.join(base_dir, split, lang), exist_ok=True)

# Create a mapping for l1 values
l1_to_int = {lang: idx for idx, lang in enumerate(languages)}

# Function to map l1 to integers for train and validation datasets
def map_l1_to_int(example):
    example["l1"] = l1_to_int.get(example["l1"], -1)  # Use -1 for unknown languages
    return example

# Apply mapping to train and validation datasets
edacc_dev = edacc_dev.map(map_l1_to_int)

# Function to save audio data and corresponding text
def save_audio(example, folder, map_l1=False):
    lang_folder = os.path.join(folder, str(example["l1"]) if map_l1 else example["l1"])  # Use mapped integer value or original language label
    os.makedirs(lang_folder, exist_ok=True)
    audio_filename = example['audio']['path'].split('/')[-1]
    audio_path = os.path.join(lang_folder, audio_filename)
    # Save audio file
    torchaudio.save(audio_path, torch.tensor(example['audio']['array']).unsqueeze(0), example['audio']['sampling_rate'])
    # Save corresponding text file
    text_path = os.path.join(lang_folder, f"{audio_filename}.txt")
    with open(text_path, "w") as f:
        f.write(example["text"])
    print(f"Saved audio: {audio_path}")
    print(f"Saved text: {text_path}")

# Save test audio and text to respective language directories with original l1 labels
print("Saving test data...")
for example in edacc_test:
    save_audio(example, os.path.join(base_dir, 'test'))

# Split the validation dataset into train and validation sets
X = range(len(edacc_dev))
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

train_dataset = edacc_dev.select(X_train)
val_dataset = edacc_dev.select(X_val)

# Save train audio and text to respective language directories with mapped l1 labels
print("Saving train data...")
for example in train_dataset:
    save_audio(example, os.path.join(base_dir, 'train'), map_l1=True)

# Save validation audio and text to respective language directories with mapped l1 labels
print("Saving validation data...")
for example in val_dataset:
    save_audio(example, os.path.join(base_dir, 'validation'), map_l1=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/9848 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9289 [00:00<?, ? examples/s]

Map:   0%|          | 0/9848 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Saved text: organized_data/train/3/EDACC-C22-203.wav.txt
Saved audio: organized_data/train/22/EDACC-C01-146.wav
Saved text: organized_data/train/22/EDACC-C01-146.wav.txt
Saved audio: organized_data/train/0/EDACC-C46_P1-29.wav
Saved text: organized_data/train/0/EDACC-C46_P1-29.wav.txt
Saved audio: organized_data/train/-1/EDACC-C40_P3-40.wav
Saved text: organized_data/train/-1/EDACC-C40_P3-40.wav.txt
Saved audio: organized_data/train/8/EDACC-C30-616.wav
Saved text: organized_data/train/8/EDACC-C30-616.wav.txt
Saved audio: organized_data/train/-1/EDACC-C40_P3-110.wav
Saved text: organized_data/train/-1/EDACC-C40_P3-110.wav.txt
Saved audio: organized_data/train/20/EDACC-C50-255.wav
Saved text: organized_data/train/20/EDACC-C50-255.wav.txt
Saved audio: organized_data/train/8/EDACC-C30-100.wav
Saved text: organized_data/train/8/EDACC-C30-100.wav.txt
Saved audio: organized_data/train/7/EDACC-C55-173.wav
Saved text: organized_data/train/7/EDAC

In [ ]:
# Model parameters
model_name_or_path = "openai/whisper-small"
task = "transcribe"
language = "en"

# Initialize feature extractor, tokenizer, processor, and model
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)
model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path)

# Function to downsample audio samples to 16 kHz
def downsample_audio(audio_array, source_sr, target_sr=16000):
    waveform = torch.tensor(audio_array, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    resampled_waveform = torchaudio.transforms.Resample(source_sr, target_sr)(waveform)
    resampled_audio = resampled_waveform.squeeze(0).numpy()  # Remove channel dimension
    return resampled_audio

# Prepare the dataset
def prepare_dataset(batch):
    audio = batch["audio"]
    if audio["sampling_rate"] != 16000:
        audio["array"] = downsample_audio(audio["array"], audio["sampling_rate"], 16000)
        audio["sampling_rate"] = 16000

    input_features = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    label_ids = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=448, return_tensors="pt").input_ids

    batch["input_features"] = input_features
    batch["labels"] = label_ids[0]
    batch["l1"] = batch["l1"]
    batch["audio"] = audio
    batch["text"] = batch["text"]

    return batch

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
# Apply preprocessing
train_data = train_dataset.map(prepare_dataset, remove_columns=["audio"])
val_data = val_dataset.map(prepare_dataset, remove_columns=["audio"])

Map:   0%|          | 0/7878 [00:00<?, ? examples/s]

Map:   0%|          | 0/1970 [00:00<?, ? examples/s]

In [ ]:
val_data

Dataset({
    features: ['speaker', 'text', 'accent', 'raw_accent', 'gender', 'l1', 'input_features', 'labels'],
    num_rows: 1970
})

In [ ]:
train_data1 = train_data.remove_columns(['speaker', 'accent', 'raw_accent', 'gender'])
val_data1 = val_data.remove_columns(['speaker', 'accent', 'raw_accent', 'gender'])

In [ ]:
train_data1

Dataset({
    features: ['text', 'l1', 'input_features', 'labels'],
    num_rows: 7878
})

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, padding="max_length", max_length=448, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch["attention_mask"].ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        batch["l1"] = torch.tensor([feature["l1"] for feature in features], dtype=torch.int64)  # Include l1 as a tensor

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
# Custom loss functions
def compute_erm_loss(outputs, labels, erm_loss_func):
    return erm_loss_func(outputs.view(-1, outputs.size(-1)), labels.view(-1))

def compute_group_dro_loss(outputs, labels, groups, erm_loss_func):
    unique_groups = groups.unique()
    group_losses = []
    for group in unique_groups:
        group_indices = (groups == group)
        group_loss = erm_loss_func(outputs[group_indices].view(-1, outputs.size(-1)), labels[group_indices].view(-1))
        group_losses.append(group_loss)
    return max(group_losses) if group_losses else torch.tensor(0.0, device=outputs.device)

def compute_spectral_norm(model):
    return sum(torch.linalg.norm(p).item() for p in model.parameters() if p.dtype.is_floating_point)

class CustomLoss(nn.Module):
    def __init__(self, model, erm_loss_func=nn.CrossEntropyLoss(), lambda_erm=1.0, lambda_dro=1.0, lambda_spectral=0.01):
        super().__init__()
        self.model = model
        self.erm_loss_func = erm_loss_func
        self.lambda_erm = lambda_erm
        self.lambda_dro = lambda_dro
        self.lambda_spectral = lambda_spectral

    def compute_spectral_decoupling_loss(self):
        spectral_norm = sum(
            torch.linalg.norm(p.data, ord=2, dim=-1).sum() for p in self.model.parameters() if p.requires_grad and p.dtype.is_floating_point
        )
        spectral_decoupling_loss = self.lambda_spectral * spectral_norm
        return spectral_decoupling_loss

    def forward(self, outputs, labels, groups):
        loss_erm = compute_erm_loss(outputs, labels, self.erm_loss_func)
        loss_dro = compute_group_dro_loss(outputs, labels, groups, self.erm_loss_func)
        spectral_decoupling_loss = self.compute_spectral_decoupling_loss()
        total_loss = (self.lambda_erm * loss_erm) + (self.lambda_dro * loss_dro) + spectral_decoupling_loss
        return total_loss

# Custom trainer
class CustomTrainer(Trainer):
    def __init__(self, custom_loss_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.custom_loss_func = custom_loss_func

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        groups = inputs.pop("l1")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.custom_loss_func(logits, labels, groups)
        return (loss, outputs) if return_outputs else loss

# Initialize the custom loss
custom_loss = CustomLoss(model)


In [ ]:
# Custom trainer
class CustomTrainer(Trainer):
    def __init__(self, custom_loss_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.custom_loss_func = custom_loss_func

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        groups = inputs.pop("l1")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.custom_loss_func(logits, labels, groups)
        return (loss, outputs) if return_outputs else loss

# Initialize the custom loss
custom_loss = CustomLoss(model)

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class CustomLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            logger.info(f"Step: {state.global_step}, Loss: {logs.get('loss')}, Learning Rate: {logs.get('learning_rate')}")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    gradient_accumulation_steps=8,
    fp16=True,
    remove_unused_columns=False,
    logging_strategy="steps",
    logging_steps=10,
)

# Clear CUDA cache
torch.cuda.empty_cache()

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data1,
    eval_dataset=val_data1,
    data_collator=data_collator,
    custom_loss_func=custom_loss,
    callbacks=[CustomLoggerCallback],
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save the model
model.save_pretrained("./whisper__fusion_saved_model")
processor.save_pretrained("./whisper__fusion_saved_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


[]

In [ ]:
# Load the WER metric
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Decode the predictions
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)

    # Decode the labels
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

<ipython-input-72-f52500aea0f8>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The repository for wer contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wer.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
import os
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from datasets import load_metric
import torchaudio

# Load the saved model and processor
model = WhisperForConditionalGeneration.from_pretrained("./whisper__fusion_saved_model")
processor = WhisperProcessor.from_pretrained("./whisper__fusion_saved_model")

# Function to downsample audio
def downsample_audio(audio_array, source_sr, target_sr=16000):
    waveform = torch.tensor(audio_array, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    resampled_waveform = torchaudio.transforms.Resample(source_sr, target_sr)(waveform)
    resampled_audio = resampled_waveform.squeeze(0).numpy()  # Remove channel dimension
    return resampled_audio

# Function to evaluate WER for each language folder
def evaluate_language_wer(folder_path, processor, model, batch_size=16):
    wer_metric = load_metric("wer", trust_remote_code=True)
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
    predictions = []
    references = []

    for i in range(0, len(audio_files), batch_size):
        batch_files = audio_files[i:i+batch_size]
        input_features = []
        for file in batch_files:
            audio_path = os.path.join(folder_path, file)
            text_path = audio_path + ".txt"
            with open(text_path, "r") as f:
                references.append(f.read().strip())
            audio, sampling_rate = torchaudio.load(audio_path)
            audio = audio.squeeze().numpy()
            if sampling_rate != 16000:
                audio = downsample_audio(audio, sampling_rate)
            features = processor.feature_extractor(audio, sampling_rate=16000, return_tensors="pt").input_features
            input_features.append(features)

        input_features = torch.cat(input_features).to(model.device)
        with torch.no_grad():
            generated_ids = model.generate(input_features, forced_bos_token_id=processor.tokenizer.pad_token_id)
        transcriptions = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend(transcriptions)

    wer = wer_metric.compute(predictions=predictions, references=references)
    return wer

# Directory where test data is organized
base_dir = "organized_data/test"
languages = os.listdir(base_dir)

# Evaluate and print WER for each language
for lang in languages:
    folder_path = os.path.join(base_dir, lang)
    wer = evaluate_language_wer(folder_path, processor, model)
    print(f"Language: {lang}, WER: {wer:.4f}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Language: 1, WER: 0.5407
Language: Mandarin, WER: 0.4320


KeyboardInterrupt: 